In [ ]:
from google.colab import drive
import json
import glob

drive.mount('/content/drive')

In [ ]:
!pip install datasets
!pip install peft

모델 파인튜닝

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 필요한 라이브러리 임포트
import json
from datasets import Dataset
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model

# 모델 경로 설정 (Google Drive 내부 경로로 변경)
model_path = "/content/drive/MyDrive/cv project/llama-3.2-Korean-Bllossom-3B"

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,  # GPU에서 효율적 연산
    device_map=None  # 직접 디바이스 지정
)
model = model.to(device)

# **여기에 pad_token 설정 추가**
tokenizer.pad_token = tokenizer.eos_token

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# 필수 키 확인 및 PROMPT 생성 함수
def create_prompt_and_instruction(json_data):
    # 기본 프롬프트 정의
    category_prompts = {
        "차트": (
            '너는 차트를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다.'
        ),
        "표": (
            '너는 표를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다.'
        ),
    }

    # 유형 키에 '표'가 포함되어 있는지 확인
    if "표" in json_data["유형"]:
        prompt = category_prompts["표"]
    else:
        prompt = category_prompts["차트"]

    # Instruction 생성
    instruction = (
        f"내용: {json_data['내용']}\n"
        "위 내용을 기반으로 요약문을 작성해줘."
    )
    return prompt, instruction

# 학습 데이터 로드
json_file_path = "/content/drive/MyDrive/cv project/기본표/학습데이터/result_table_with_summary.json"
json_file = load_json(json_file_path)

# 데이터 준비
data_list = []

for index, json_data in enumerate(json_file):
    # 필수 키 확인
    required_keys = ["data_id", "제목", "유형", "내용", "요약"]
    if not all(key in json_data for key in required_keys):
        print(f"[Error] JSON 항목 {index}가 필수 키 {required_keys}를 포함하지 않습니다.")
        continue

    # PROMPT 및 instruction 생성
    PROMPT, instruction = create_prompt_and_instruction(json_data)

    # 입력 텍스트와 대상 텍스트 준비
    input_text = f"{PROMPT}\n{instruction}\n"
    target_text = json_data["요약"]

    # 데이터 리스트에 추가
    data_list.append({
        'input_text': input_text,
        'target_text': target_text
    })

# Dataset 생성
from datasets import Dataset

dataset = Dataset.from_list(data_list)

# 데이터 전처리 함수 정의
def preprocess_function(examples):
    input_texts = examples['input_text']
    target_texts = examples['target_text']
    full_texts = [input_texts[i] + target_texts[i] for i in range(len(input_texts))]

    # 토크나이저 설정: padding=True로 변경
    tokenized_inputs = tokenizer(
        input_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    tokenized_full = tokenizer(
        full_texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

    labels = tokenized_full["input_ids"].clone()

    # 입력 부분은 -100으로 마스킹
    for i in range(len(input_texts)):
        input_len = tokenized_inputs['input_ids'].shape[1]
        labels[i, :input_len] = -100

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 데이터셋 전처리
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

# LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/cv project/finetuningllama",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=1,
    logging_steps=10,
    save_steps=200,
    evaluation_strategy="no"
)

# DataCollatorWithPadding 사용
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, padding=True)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 모델 학습
trainer.train()

# 파인튜닝된 모델 저장
trainer.save_model("/content/drive/MyDrive/cv project/finetuningllama")

print("파인튜닝된 모델이 '/content/drive/MyDrive/cv project/finetuningllama'에 저장되었습니다.")


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 모델 경로 설정
original_model_path = "/content/drive/MyDrive/cv project/llama-3.2-Korean-Bllossom-3B"
finetuned_model_path = "/content/drive/MyDrive/cv project/finetuningllama"

# GPU 디바이스 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 토크나이저를 원본 모델 경로에서 로드
tokenizer = AutoTokenizer.from_pretrained(original_model_path)
tokenizer.pad_token = tokenizer.eos_token  # pad_token 설정

# 모델을 원본 모델 경로에서 로드
model = AutoModelForCausalLM.from_pretrained(
    original_model_path,
    torch_dtype=torch.bfloat16,  # GPU에서 효율적 연산
    device_map=None  # 직접 디바이스 지정
)
model = model.to(device)
model.config.pad_token_id = tokenizer.pad_token_id

# 파인튜닝된 모델 로드 (LoRA 가중치 적용)
model = PeftModel.from_pretrained(model, finetuned_model_path)

# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# JSON 파일 저장 함수
def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Google Drive 내 JSON 파일 경로
json_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure.json"  # 입력 JSON 파일 경로
output_file_path = "/content/drive/MyDrive/cv project/기본표/result_figure_with_summary_finetuning.json"  # 저장 경로

# JSON 데이터 로드
json_file = load_json(json_file_path)

# 필수 키 확인 및 PROMPT 생성 함수 (이전 코드와 동일)
def create_prompt_and_instruction(json_data):
    # 기본 프롬프트 정의
    category_prompts = {
        "차트": (
            '너는 차트를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 차트 정보를 정리해서 하나의 문장으로 요약문을 생성하라. 차근차근 생각해보자.'
            '데이터에서 패턴, 공통점, 차이점, 이상치나 중요한 점이 있다면 이를 포함하라.'
        ),
        "표": (
            '너는 표를 분석하여 명확하고 객관적인 요약문을 생성하는 AI이다. 아래의 표 정보를 정리해서 하나의 문장으로 요약문을 생성하라. 차근차근 생각해보자.'
        ),
    }

    # 유형 키에 '표'가 포함되어 있는지 확인
    if "표" in json_data["유형"]:
        prompt = category_prompts["표"]
    else:
        prompt = category_prompts["차트"]

    # Instruction 생성
    instruction = (
        f"다음은 {json_data['제목']}에 대한 설명입니다.\n"
        f"유형: {json_data['유형']}\n"
        f"내용: {json_data['내용']}\n"
        "위 내용을 기반으로 요약문을 작성해줘."
    )
    return prompt, instruction

# JSON 파일의 모든 항목에 대해 실행
for index, json_data in enumerate(json_file):
    # 필수 키 확인
    required_keys = ["data_id", "제목", "유형", "내용"]
    if not all(key in json_data for key in required_keys):
        print(f"[Error] JSON 항목 {index}가 필수 키 {required_keys}를 포함하지 않습니다.")
        continue

    # PROMPT 및 instruction 생성
    PROMPT, instruction = create_prompt_and_instruction(json_data)

    # 입력 텍스트 생성
    input_text = f"{PROMPT}\n{instruction}\n"

    # 토크나이즈 및 텐서 변환
    input_ids = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    # 텍스트 생성
    outputs = model.generate(
        input_ids=input_ids['input_ids'],
        max_new_tokens=512,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.1,
        top_p=0.9
    )

    # 결과 텍스트 생성
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 입력 텍스트 부분을 제거하여 생성된 요약만 추출
    generated_summary = generated_text[len(input_text):].strip()

    print(f"[Result for JSON {index}]")
    print(generated_summary)
    print("\n" + "="*80 + "\n")

    # JSON 데이터에 요약 추가
    json_data["요약"] = generated_summary

# 업데이트된 JSON 파일 저장
save_json(output_file_path, json_file)

print(f"업데이트된 JSON 파일이 {output_file_path}에 저장되었습니다.")
